In [36]:
import tensorflow.keras as keras
import json
from preprocess_module import SEQUENCE_LENGTH, MAPPING_PATH
import numpy as np
import music21 as m21

In [ ]:

class Melody_generator():
    def __init__(self, model_path = "model.h5"):
        self.model_path = model_path
        self.model = keras.models.load_model(model_path)
        self._start_symbols = ["/"] * SEQUENCE_LENGTH #indicador de inicio de nueva cancion        
        
        #loading the dictionary of mappings..
        with open(MAPPING_PATH, "r") as fp:
            self._mappings = json.load(fp)
            
    
    def generate_melody(self, seed, num_steps, max_sequence_length, temperature):
        """seed: trozo de lmelodia que sera el input para que el modelo
        le genere una continuacion. Ej: 64_63_ _ r_47
        num_steps: largo que va a tener esta continuacion
        que la red va a generar como output de cada input 
        max_sequence_length: maximo largo que puede tener un input (seed). En este
        caso será 64 steps o 4 compases de 4/4
        temperature: xxxx"""
        #Create a seed with a start symbol
        seed = seed.split() #convierto seed a una lista
        #creo una lista contenedora donde se guardara la melodia que se genere
        #Parte como siempre con la secuencia de inicio..
        melody = seed
        seed = self._start_symbols + seed
        
        
        #Map seed to integers (translate to integers using the vocabulary)
        seed = [self._mappings[symbol]  for symbol in seed]   
        
        for _ in range(num_steps):
            #limit the seed to max_sequence_length
            #toma los ultimos max_sequence_length steps (en este caso 64)
            seed = seed[-max_sequence_length:]
            #one hot encode the seed
            one_hot_seed = keras.utils.to_categorical(seed, num_classes= len(self._mappings))
            #hay que modificar el shape para Keras como tensor (1, max_sequence_length, num symbols on vocabulary)
            one_hot_seed = one_hot_seed[np.newaxis, ...] #crea una dimension adicional y deja el resto igual
            
            #make a prediction
            #Usando Keras se obtiene un vector de distribucion de probabilidades
            probabilities = self.model.predict(one_hot_seed)[0]
            #lo mas obvio es tomar el indice de mayor prob para predecir, pero
            #se usara temperature para obtener un mejor resultado
            output_int = self._sample_with_temperature(probabilities, temperature)
            
            #Ahora actualizo el seed de manera que incorpore la nota/simbolo que acaba de predecir
            seed.append(output_int)
            
            #Obtengo el simbolo original asociado a este output_int que acabo de predecir
            output_symbol = [k for k,v in self._mappings.items() if v == output_int][0]
            
            #Check if we are ate the end of the melody/beginnig of the next else we continue
            if output_symbol == "/":
                break
                
            else: #update the melody
                melody.append(output_symbol)
        return melody
            
      
        
            
        
    def _sample_with_temperature(self, probabilities, temperature):
        """if temperature = infinity---> al probabilities are equal, son we could
        randomly pick any option 
        if temperature close to zero: prob ---> the higher value in probabilities vector
        is assigned a one (deterministic)
        if temperature close to zero: prob ---> the higher value in probabilities vector
        if temperature = 1 --> vector de probabilidades se deja tal cual
        Por tanto, a mayor temperatura más random es la elección y mas exploratoria
        la busqueda 
        """
        #primero uso softmax para que me quede un vector de probabilidades (suma 1)
        predictcions = np.log(probabilities) / temperature
        probabilities = np.exp(predictcions) / np.sum(np.exp(predictcions))
        #hago que se escoja una opcion random pero dandole mas probabilidad segun su distribucion
        choices = range(len(probabilities))
        index = np.random.choice(choices, p = probabilities)
        
        return index
        
    def save_into_midi(self, melody, step_duration = 0.25, format = "midi",file_name = "melody.midi"):
        # converts the predicted melody into a midi file
    
        #create a music21 stream: un contenedor para todos los componentes de la melodia
        
        stream = m21.stream.Stream()
        
        
        #parse all the symbols in the melody and create note/rest music21 objects for filling the stream
        
        start_symbol = None
        step_counter = 1 
        
        for i, symbol in enumerate(melody):
            
            #Si el simbolo es una nota o rest se continua leyendo...
            if symbol !="_" or i +1 == len(melody): #si llegamos a una nora/descanso o al final de la melodia
               
                if start_symbol is not None: #make sure we´re not in the first symbol of the melody
                    quarter_length_duration = step_duration * step_counter #cuantos 0.25 se han recorrido
                    
                    #case 1: rest
                    if start_symbol == "r":
                        m21_event = m21.note.Rest(quarterLength = quarter_length_duration) #duracion del silencio
                        
                    #case 2: note
                    else:
                        m21_event = m21.note.Note(int(start_symbol), quarterLength=quarter_length_duration) #guarda la nota con su pitch
                        
                    
                    stream.append(m21_event)
                    
                    #reset step counter
                    step_counter = 1
                    
                    #update the start symbol
                start_symbol = symbol
                    
            #si el simbolo es una prolongacion "_", se suma 1 al counter
            else:
                step_counter +=1 
                
            
                
        
        
        #write the m21 stream to a midi file
        
        stream.write(format, file_name)
        
    
    
    
if __name__ == "__main__":
    mg = Melody_generator()
    "67 _ _ _ _ _ 65 _ 64 _ 62 _ 60 _ _ _"
    melody = mg.generate_melody(seed,500,SEQUENCE_LENGTH,0.3 )
    print(len(melody))
    mg.save_into_midi(melody)
    print("Done!")
    
    
    ###REVISAR Y COMPARAR CODIGO + DATA
    

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


/Users/mauricioalfaro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in log
